CNN = Convnet = Convolution Neural Network

## CNN 구성

- convolution : 위치와 상관없이 특성 파악
    - local, shared
     -locally connected : 위치에 상관있음 -> data augmentation 필수
- pooling : 특정 영역을 하나의 값으로 표현
    - network는 줄어들면서 특징은 살림.
- fully connected
    - global average pooling
- activation function
    - nonlinearity : 데이터의 분포가 바뀌면서 새로운 분포 생성
- normalization
    - input
    - batch normalization
    
<br></br>

---

논문 : Identity Mappings in Deep Residual Networks

- 논문에서 layer깊게 쌓은 것과 비교했을 때 test error뿐만 아니라 train error도 같이 올라가는 문제가 생김
    > overfitting문제가 아닐 수도! underfitting문제일 수도 있다.

<br></br>



## Resnet

- **short cut connection** : 입력값과 결과값을 합쳐서 다음 layer의 input으로 집어넣음(residual block)
- batch normalization
- googlenet 영향으로 1 * 1 conv씀
- layer를 152개 만듬 -> 잘됨
- PRelu : intialize를 gaussian -> xaiver -> he


SOTA(State of the art) : 현존 최고의 기술

In [1]:
import tensorflow as tf

# tf.keras.layers.LocallyConnected2D : 위치에 상관이 있음.

C:\Users\Gyu\Anaconda3\envs\py37\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\Gyu\Anaconda3\envs\py37\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
C:\Users\Gyu\Anaconda3\envs\py37\lib\site-packages\numpy\.libs\libopenblas.TXA6YQSD3GCQQC22GEQ54J2UDCXDXHWN.gfortran-win_amd64.dll
  stacklevel=1)


## Transfer learning

- 만들어 놓은 model 사용

1. tf.keras.application

In [5]:
vgg = tf.keras.applications.VGG16()
vgg.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [29]:
resnet = tf.keras.applications.ResNet50()

In [21]:
# resnet.summary()
    # connection

In [30]:
tf.keras.utils.plot_model(resnet)

('Failed to import pydot. You must `pip install pydot` and install graphviz (https://graphviz.gitlab.io/download/), ', 'for `pydotprint` to work.')


In [34]:
resnet.get_layer('conv5_block1_3_conv')

In [32]:
len(resnet.get_layer('conv5_block3_3_bn').weights)

4

In [31]:
resnet.get_layer('conv5_block3_3_bn').output

<tf.Tensor 'conv5_block3_3_bn/cond_2/Identity:0' shape=(None, 7, 7, 2048) dtype=float32>

- tensorflow deeplearning 모델 만들기
    - tf.nn
    - estimator
    - keras(sequential, model, model 상속)
    
- 모델을 상속받아 사용가능

<br>

- efficient : resnet과 성능 비슷하지만 간단
- mobilenet
- 복잡한 모델이 나온 이후로 성능이 비슷한 간단한 모델 구축

In [22]:
inputs = tf.keras.Input(shape=(28,28))
layer1 = tf.keras.layers.Dense(64)(inputs)
layer2 = tf.keras.layers.Dense(64)(inputs)

In [23]:
layer3 = tf.keras.layers.Dense(64)(layer1)
layer4 = tf.keras.layers.Add()([layer1, layer3])
# 설명보면 값을 list로 받는다고 나와있음.

In [24]:
model = tf.keras.models.Model(inputs=inputs, outputs=layer4 )

In [25]:
model.summary()
# skip connection이 이루어졌음

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 28, 28)]     0                                            
__________________________________________________________________________________________________
dense (Dense)                   (None, 28, 64)       1856        input_4[0][0]                    
__________________________________________________________________________________________________
dense_2 (Dense)                 (None, 28, 64)       4160        dense[0][0]                      
__________________________________________________________________________________________________
add (Add)                       (None, 28, 64)       0           dense[0][0]                      
                                                                 dense_2[0][0]         

In [26]:
tf.keras.utils.plot_model(model)

('Failed to import pydot. You must `pip install pydot` and install graphviz (https://graphviz.gitlab.io/download/), ', 'for `pydotprint` to work.')


#### 여러 layer에서 온 결과값 연산
    - tf.keras.layers.Add
    - tf.keras.layers.Average
    - tf.keras.layers.Concatenate
    - tf.keras.layers.Lambda
    - tf.keras.layers.Multiply
    - tf.keras.layers.Subtract 

<br>

---

## Batch Normalization

- 기본적으로 relu와 adam쓰면 학습할 수 있다.
    
    > but layer가 깊어지면 안됨!
    
- weight bias가 다양하게 바뀜
    -> forward 방향으로 진행되면서 앞의 잘못으로 뒤의 layer에서 더 큰 잘못으로 잘못된 분포를 나오게 함
        -> 학습이 안되는 경우가 생김!
        
- minmaxscaling
- SGD -> mini-batch
- 학습되어서 나오는 결과를 normalization -> 분포가 달라지지 않을 것임
    -> batch를 normalize하여 학습이 안정적, 성능도 좋음
    - learning rate도 크게 잡을 수 있다.
    - regularization -> dropout효과

<br></br>

- batch normalization은 가장 범용적인 기법이라 할 수 있음!
- 특성을 유지한 채로 값 범위를 공통 스케일로 변경
    - 스케일이 다르면 고려할 것이 많아짐 -> 학습이 잘 안됨.
    - 공통 스케일로 맞춰주면 고려할 것이 적어짐  
- 초기값이 잘못되어도 괜찮은 편
- dropout효과
- test할 때는 overhead가 없어 실제에 좋음.
- 그렇지만 이론적으로 증명이 잘 안됨.


### Normalization, Regularization, Standardization
- Regularization : overfitting 줄여 일반화된 모델을 만들기 위함.
- Standardization : z-score

<br>

#### Internal Covariance shift
- input의 distribution이 달라지는 현상
- network는 계층적 구조 -> 어느 시점에서 잘못되면 뒤로도 계속 잘못됨.
- 어떤 범위에서 범위 이상 넘어가지 않게 막음 -> 리피츠 연속함수
- 많이 deep하면 gradient vanishing
     - variance가 커져 값이 제대로 전달되지 못할 수도 있다.

**Whitening**
- 평균0, 분산 1로 만들고 uncorrelated하게 만들어줌.
- 그렇지만 whitening해서 0에 가까우면 영향력을 못 가지는 문제가 생김(bias가 없어지는 문제)

-> mini-batch 학습 : batch size 별로 사용
    - ex) 256개 평균 내서 분산 계산 후 표준화(앱실론으로 stability높임)
        - 최종적으로 **shift(다른 factor를 곱함)**
- batchsize에서 나온 것으로 대표값을 취하는 식
    -> batch size가 크면 클수록 batch normalization이 잘 됨.
    
- 학습을 통해 shift parameter 정의

</br>
test 할 때는....?
- 분산을 예측하는 방식을 씀
    - 이동 평균 개념을 통해 학습할 때 썼던 평균 분산을 가져옴
    - 실제는 학습하지 않으므로 layer를 갖다 붙이는 등
    
**Activation 전에 넣어서**

tf.keras.layers.Dense(64,activation='relu')

    > tf.keras.layers.Dense(64)
    > tf.keras.layers.BatchNormalization()
    > tf.keras.layers.Activation('relu')

In [27]:
tf.keras.layers.BatchNormalization()

tf.keras.layers.Activation('relu')

'brew'은(는) 내부 또는 외부 명령, 실행할 수 있는 프로그램, 또는
배치 파일이 아닙니다.
